| [03_data_science/08_Pandas数据操作.ipynb](https://github.com/shibing624/python-tutorial/blob/master/03_data_science/08_Pandas数据操作.ipynb)  | Pandas操作  |[Open In Colab](https://colab.research.google.com/github/shibing624/python-tutorial/blob/master/03_data_science/08_Pandas数据操作.ipynb) |

# 3. Pandas数据操作

在第一部分的基础上，数据会有更多种操作方式：

* 通过列名、行index来取数据，结合ix、iloc灵活的获取数据的一个子集（第一部分已经介绍）
* 按记录拼接（就像Union All）或者关联（join）
* 方便的自定义函数映射
* 排序
* 缺失值处理
* 与Excel一样灵活的数据透视表（在第四部分更详细介绍）

### 3.1 数据整合：方便灵活

### 3.1.1 横向拼接：直接DataFrame

In [1]:
import numpy as np
import pandas as pd
pd.DataFrame([np.random.rand(2), np.random.rand(2), np.random.rand(2)], columns=['C1', 'C2'])

,C1,C2
0,0.838146,0.470503
1,0.107491,0.243259
2,0.604792,0.652182


### 3.1.2 横向拼接：Concatenate

In [2]:
import json
json_data = [{'name': 'Wang', 'sal': 50000, 'job': 'VP'},
             {'name': 'Zhang', 'job': 'Manager', 'report': 'VP'},
             {'name': 'Li', 'sal': 5000, 'report': 'Manager'}]
data_employee = pd.read_json(json.dumps(json_data))
data_employee_ri = data_employee.reindex(columns=['name', 'job', 'sal', 'report'])

pd.concat([data_employee_ri, data_employee_ri, data_employee_ri])

,name,job,sal,report
0,Wang,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP
2,Li,NaN,5000.0,Manager
0,Wang,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP
2,Li,NaN,5000.0,Manager
0,Wang,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP
2,Li,NaN,5000.0,Manager


In [3]:
pd.concat([data_employee_ri, data_employee_ri, data_employee_ri],ignore_index=True)

,name,job,sal,report
0,Wang,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP
2,Li,NaN,5000.0,Manager
3,Wang,VP,50000.0,NaN
4,Zhang,Manager,NaN,VP
5,Li,NaN,5000.0,Manager
6,Wang,VP,50000.0,NaN
7,Zhang,Manager,NaN,VP
8,Li,NaN,5000.0,Manager


### 3.1.3 纵向拼接：Merge

根据数据列关联，使用on关键字

* 可以指定一列或多列
* 可以使用left_on和right_on

In [4]:
pd.merge(data_employee_ri, data_employee_ri, on='name')

,name,job_x,sal_x,report_x,job_y,sal_y,report_y
0,Wang,VP,50000.0,NaN,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP,Manager,NaN,VP
2,Li,NaN,5000.0,Manager,NaN,5000.0,Manager


In [5]:
pd.merge(data_employee_ri, data_employee_ri, on=['name', 'job'])

,name,job,sal_x,report_x,sal_y,report_y
0,Wang,VP,50000.0,NaN,50000.0,NaN
1,Zhang,Manager,NaN,VP,NaN,VP
2,Li,NaN,5000.0,Manager,5000.0,Manager


根据index关联，可以直接使用left_index和right_index

In [6]:
data_employee_ri.index.name = 'index1'
pd.merge(data_employee_ri, data_employee_ri, left_index=True, right_index=True)

,name_x,job_x,sal_x,report_x,name_y,job_y,sal_y,report_y
index1,,,,,,,,
0,Wang,VP,50000.0,NaN,Wang,VP,50000.0,NaN
1,Zhang,Manager,NaN,VP,Zhang,Manager,NaN,VP
2,Li,NaN,5000.0,Manager,Li,NaN,5000.0,Manager


TIPS: 增加how关键字，并指定
* how = 'inner'
* how = 'left'
* how = 'right'
* how = 'outer'

结合how，可以看到merge基本再现了SQL应有的功能，并保持代码整洁。

In [7]:
df31_a = pd.DataFrame({'name':['老王', '老张', '老李'], 'sal':[5000, 3000, 1000]})
df31_a

,name,sal
0,老王,5000
1,老张,3000
2,老李,1000


In [8]:
df31_b = pd.DataFrame({'name':['老王', '老刘'], 'job':['VP', 'Manager']})
df31_b

,name,job
0,老王,VP
1,老刘,Manager


how='left': 保留左表信息

In [9]:
pd.merge(df31_a, df31_b, on='name', how='left')

,name,sal,job
0,老王,5000,VP
1,老张,3000,NaN
2,老李,1000,NaN


how='right': 保留右表信息

In [10]:
pd.merge(df31_a, df31_b, on='name', how='right')

,name,sal,job
0,老王,5000.0,VP
1,老刘,NaN,Manager


how='inner': 保留两表交集信息，这样尽量避免出现缺失值

In [11]:
pd.merge(df31_a, df31_b, on='name', how='inner')

,name,sal,job
0,老王,5000,VP


how='outer': 保留两表并集信息，这样会导致缺失值，但最大程度的整合了已有信息

In [12]:
pd.merge(df31_a, df31_b, on='name', how='outer')

,name,sal,job
0,老王,5000.0,VP
1,老张,3000.0,NaN
2,老李,1000.0,NaN
3,老刘,NaN,Manager


### 3.2 数据清洗三剑客

接下来的三个功能，**map**,**applymap**,**apply**,功能，是绝大多数数据分析师在数据清洗这一步骤中的必经之路。

他们分别回答了以下问题：

* 我想根据一列数据新做一列数据，怎么办？（Series->Series）
* 我想根据整张表的数据新做整张表，怎么办？ （DataFrame->DataFrame）
* 我想根据很多列的数据新做一列数据，怎么办？ （DataFrame->Series）

不要再写什么for循环了！改变思维，提高编码和执行效率

In [13]:
data_np = np.asarray([('Japan', 'Tokyo', 4000),
                      ('Korea', 'Seoul', 1300),
                      ('China', 'Beijing', 9100)])
df32 = pd.DataFrame(data_np, columns=['nation', 'capital', 'GDP'])
df32

,nation,capital,GDP
0,Japan,Tokyo,4000
1,Korea,Seoul,1300
2,China,Beijing,9100


### map: 以相同规则将一列数据作一个映射，也就是进行相同函数的处理

In [14]:
def GDP_Factorize(v):
    fv = np.float64(v)
    if fv > 6000.0:
        return 'High'
    elif fv < 2000.0:
        return 'Low'
    else:
        return 'Medium'

df32['GDP_Level'] = df32['GDP'].map(GDP_Factorize)
df32['NATION'] = df32.nation.map(str.upper)
df32

,nation,capital,GDP,GDP_Level,NATION
0,Japan,Tokyo,4000,Medium,JAPAN
1,Korea,Seoul,1300,Low,KOREA
2,China,Beijing,9100,High,CHINA


### 类似的功能还有applymap，可以对一个dataframe里面每一个元素像map那样全局操作

In [15]:
df32.applymap(lambda x: float(x)*2 if x.isdigit() else x.upper())

,nation,capital,GDP,GDP_Level,NATION
0,JAPAN,TOKYO,8000.0,MEDIUM,JAPAN
1,KOREA,SEOUL,2600.0,LOW,KOREA
2,CHINA,BEIJING,18200.0,HIGH,CHINA


### apply则可以对一个DataFrame操作得到一个Series

他会有点像我们后面介绍的agg,但是apply可以按行操作和按列操作，用axis控制即可。

In [16]:
df32.apply(lambda x: x['nation'] + x['capital'] + '_' + x['GDP'], axis=1)

0      JapanTokyo_4000
1      KoreaSeoul_1300
2    ChinaBeijing_9100
dtype: object

### 3.3 数据排序

* sort: 按一列或者多列的值进行行级排序
* sort_index: 根据index里的取值进行排序，而且可以根据axis决定是重排行还是列

In [17]:
data_np = np.asarray([('Japan', 'Tokyo', 4000),
                      ('Korea', 'Seoul', 1300),
                      ('China', 'Beijing', 9100)])
df33 = pd.DataFrame(data_np, columns=['nation', 'capital', 'GDP'])
df33

,nation,capital,GDP
0,Japan,Tokyo,4000
1,Korea,Seoul,1300
2,China,Beijing,9100


In [18]:
df33.sort_values(['capital', 'nation'])

,nation,capital,GDP
2,China,Beijing,9100
1,Korea,Seoul,1300
0,Japan,Tokyo,4000


In [19]:
df33.sort_values('GDP', ascending=False)

,nation,capital,GDP
2,China,Beijing,9100
0,Japan,Tokyo,4000
1,Korea,Seoul,1300


In [20]:
df33.sort_index(axis=1, ascending=True)

,GDP,capital,nation
0,4000,Tokyo,Japan
1,1300,Seoul,Korea
2,9100,Beijing,China


一个好用的功能：Rank

In [21]:
df33

,nation,capital,GDP
0,Japan,Tokyo,4000
1,Korea,Seoul,1300
2,China,Beijing,9100


In [22]:
df33.rank()

,nation,capital,GDP
0,2.0,3.0,2.0
1,3.0,2.0,1.0
2,1.0,1.0,3.0


In [23]:
df33.rank(ascending=False)

,nation,capital,GDP
0,2.0,1.0,2.0
1,1.0,2.0,3.0
2,3.0,3.0,1.0


注意tied data（相同值）的处理：
* method = 'average'
* method = 'min'
* method = 'max'
* method = 'first'

In [24]:
df33x = pd.DataFrame({'name': ['老王', '老张', '老李', '老刘'],
                      'sal': np.array([5000, 3000, 5000, 9000])})
df33x

,name,sal
0,老王,5000
1,老张,3000
2,老李,5000
3,老刘,9000


df33x.rank()默认使用method='average'，两条数据相等时，处理排名时大家都用平均值

In [25]:
df33x.sal.rank()

0    2.5
1    1.0
2    2.5
3    4.0
Name: sal, dtype: float64

method='min'，处理排名时大家都用最小值

In [26]:
df33x.sal.rank(method='min')

0    2.0
1    1.0
2    2.0
3    4.0
Name: sal, dtype: float64

method='max'，处理排名时大家都用最大值

In [27]:
df33x.sal.rank(method='max')

0    3.0
1    1.0
2    3.0
3    4.0
Name: sal, dtype: float64

method='first'，处理排名时谁先出现就先给谁较小的数值。

In [28]:
df33x.sal.rank(method='first')

0    2.0
1    1.0
2    3.0
3    4.0
Name: sal, dtype: float64

### 3.4 缺失数据处理

In [29]:
i = pd.Index([('Row_' + str(x), 'Col_' + str(y + 1)) for x in range(5) for y in range(x)])
data_multi = pd.Series(np.arange(10), index=i)
df34 = data_multi.unstack()
df34

,Col_1,Col_2,Col_3,Col_4
Row_1,0.0,NaN,NaN,NaN
Row_2,1.0,2.0,NaN,NaN
Row_3,3.0,4.0,5.0,NaN
Row_4,6.0,7.0,8.0,9.0


忽略缺失值：

In [30]:
df34.mean(skipna=True)

Col_1    2.500000
Col_2    4.333333
Col_3    6.500000
Col_4    9.000000
dtype: float64

In [31]:
df34.mean(skipna=False)

Col_1    2.5
Col_2    NaN
Col_3    NaN
Col_4    NaN
dtype: float64

如果不想忽略缺失值的话，就需要祭出fillna了：

In [32]:
df34

,Col_1,Col_2,Col_3,Col_4
Row_1,0.0,NaN,NaN,NaN
Row_2,1.0,2.0,NaN,NaN
Row_3,3.0,4.0,5.0,NaN
Row_4,6.0,7.0,8.0,9.0


In [33]:
df34.fillna(0).mean(axis=1, skipna=False)

Row_1    0.00
Row_2    0.75
Row_3    3.00
Row_4    7.50
dtype: float64

# 4. “一组”大熊猫：Pandas的groupby

groupby的功能类似SQL的group by关键字：

Split-Apply-Combine

* Split，就是按照规则分组
* Apply，通过一定的agg函数来获得输入pd.Series返回一个值的效果
* Combine，把结果收集起来

Pandas的groupby的灵活性：

* 分组的关键字可以来自于index，也可以来自于真实的列数据
* 分组规则可以通过一列或者多列

分组的具体逻辑

In [34]:
iris_file = '../data/numpy/iris.data.txt'
cnames = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width', 'class']

irisdata = pd.read_csv(iris_file, names=cnames, encoding='utf-8')
irisdata

,sepal_length,sepal_width,petal_length,petal_width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


In [35]:
irisdata_group = irisdata.groupby('class')
irisdata_group

In [36]:
for level, subsetDF in irisdata_group:
    print(level)
    print(subsetDF[::20])

Iris-setosa
    sepal_length  sepal_width  petal_length  petal_width        class
0            5.1          3.5           1.4          0.2  Iris-setosa
20           5.4          3.4           1.7          0.2  Iris-setosa
40           5.0          3.5           1.3          0.3  Iris-setosa
Iris-versicolor
    sepal_length  sepal_width  petal_length  petal_width            class
50           7.0          3.2           4.7          1.4  Iris-versicolor
70           5.9          3.2           4.8          1.8  Iris-versicolor
90           5.5          2.6           4.4          1.2  Iris-versicolor
Iris-virginica
     sepal_length  sepal_width  petal_length  petal_width           class
100           6.3          3.3           6.0          2.5  Iris-virginica
120           6.9          3.2           5.7          2.3  Iris-virginica
140           6.7          3.1           5.6          2.4  Iris-virginica


分组可以快速实现MapReduce的逻辑

* Map: 指定分组的列标签，不同的值就会被扔到不同的分组处理
* Reduce: 输入多个值，返回一个值，一般可以通过agg实现，agg能接受一个函数

In [37]:
irisdata.groupby('class').agg(lambda x:((x - x.mean())**3).sum() * (len(x) - 0.0) / 
                (len(x) - 1.0) / (len(x) - 2.0) / (x.std() * np.sqrt((len(x) - 0.0) / 
                                                                     (len(x)-1.0)))**3 if len(x) > 2 else None)

,sepal_length,sepal_width,petal_length,petal_width
class,,,,
Iris-setosa,0.116502,0.103857,0.069702,1.161506
Iris-versicolor,0.102232,-0.352014,-0.588404,-0.030249
Iris-virginica,0.114492,0.355026,0.533044,-0.125612


In [38]:
import scipy.stats
irisdata.groupby('class').agg(scipy.stats.skew)

,sepal_length,sepal_width,petal_length,petal_width
class,,,,
Iris-setosa,0.116454,0.103814,0.069673,1.161022
Iris-versicolor,0.102190,-0.351867,-0.588159,-0.030236
Iris-virginica,0.114445,0.354878,0.532822,-0.125560


#### 汇总之后的广播操作

在OLAP数据库上，为了避免groupby+join的二次操作，提出了sum()over(partition by)的开窗操作。

在Pandas中，这种操作能够进一步被transform所取代。

In [39]:
pd.concat([irisdata, irisdata.groupby('class').transform('mean')], axis=1)[::20]

,sepal_length,sepal_width,petal_length,petal_width,class,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2,Iris-setosa,5.006,3.418,1.464,0.244
20,5.4,3.4,1.7,0.2,Iris-setosa,5.006,3.418,1.464,0.244
40,5.0,3.5,1.3,0.3,Iris-setosa,5.006,3.418,1.464,0.244
60,5.0,2.0,3.5,1.0,Iris-versicolor,5.936,2.770,4.260,1.326
80,5.5,2.4,3.8,1.1,Iris-versicolor,5.936,2.770,4.260,1.326
100,6.3,3.3,6.0,2.5,Iris-virginica,6.588,2.974,5.552,2.026
120,6.9,3.2,5.7,2.3,Iris-virginica,6.588,2.974,5.552,2.026
140,6.7,3.1,5.6,2.4,Iris-virginica,6.588,2.974,5.552,2.026


#### 产生 MultiIndex（多列分组）后的数据透视表操作

一般来说，多列groupby的一个副作用就是.groupby().agg()之后你的行index已经变成了一个多列分组的分级索引。

如果我们希望达到Excel的数据透视表的效果，行和列的索引自由交换，达到统计目的，究竟应该怎么办呢？

In [40]:
factor1 = np.random.randint(0, 3, 50)
factor2 = np.random.randint(0, 2, 50)
factor3 = np.random.randint(0, 3, 50)
values = np.random.randn(50)

In [49]:
hierindexDF = pd.DataFrame({'F1': factor1, 'F2': factor2, 'F3': factor3, 'F4': values})
hierindexDF.tail()

,F1,F2,F3,F4
45,1,1,2,0.183978
46,0,0,2,1.251493
47,0,0,1,-0.424046
48,2,0,0,-1.276174
49,2,1,0,-1.239284


In [42]:
hierindexDF_gbsum = hierindexDF.groupby(['F1', 'F2', 'F3']).sum()
hierindexDF_gbsum

F4
F1 F2 F3          
0  0  0   2.074013
      1   1.849194
      2   1.998952
   1  0   0.159661
      1  -1.349574
      2  -0.687127
1  0  0   0.049578
      1  -1.997105
      2  -0.818087
   1  0   0.768738
      1   2.950726
      2  -0.846996
2  0  0  -0.952434
      1  -2.751583
   1  0   1.744497
      1   0.923548
      2   1.856369

观察Index：

In [43]:
hierindexDF_gbsum.index

MultiIndex([(0, 0, 0),
            (0, 0, 1),
            (0, 0, 2),
            (0, 1, 0),
            (0, 1, 1),
            (0, 1, 2),
            (1, 0, 0),
            (1, 0, 1),
            (1, 0, 2),
            (1, 1, 0),
            (1, 1, 1),
            (1, 1, 2),
            (2, 0, 0),
            (2, 0, 1),
            (2, 1, 0),
            (2, 1, 1),
            (2, 1, 2)],
           names=['F1', 'F2', 'F3'])

unstack：

* 无参数时，把最末index置换到column上
* 有数字参数时，把指定位置的index置换到column上
* 有列表参数时，依次把特定位置的index置换到column上

In [44]:
hierindexDF_gbsum.unstack()

F4                    
F3            0         1         2
F1 F2                              
0  0   2.074013  1.849194  1.998952
   1   0.159661 -1.349574 -0.687127
1  0   0.049578 -1.997105 -0.818087
   1   0.768738  2.950726 -0.846996
2  0  -0.952434 -2.751583       NaN
   1   1.744497  0.923548  1.856369

In [45]:
hierindexDF_gbsum.unstack(0)

F4                    
F1            0         1         2
F2 F3                              
0  0   2.074013  0.049578 -0.952434
   1   1.849194 -1.997105 -2.751583
   2   1.998952 -0.818087       NaN
1  0   0.159661  0.768738  1.744497
   1  -1.349574  2.950726  0.923548
   2  -0.687127 -0.846996  1.856369

In [46]:
hierindexDF_gbsum.unstack(1)

F4          
F2            0         1
F1 F3                    
0  0   2.074013  0.159661
   1   1.849194 -1.349574
   2   1.998952 -0.687127
1  0   0.049578  0.768738
   1  -1.997105  2.950726
   2  -0.818087 -0.846996
2  0  -0.952434  1.744497
   1  -2.751583  0.923548
   2        NaN  1.856369

In [47]:
hierindexDF_gbsum.unstack([2,0])

F4                                                              \
F3         0         1         2         0         1         2         0   
F1         0         0         0         1         1         1         2   
F2                                                                         
0   2.074013  1.849194  1.998952  0.049578 -1.997105 -0.818087 -0.952434   
1   0.159661 -1.349574 -0.687127  0.768738  2.950726 -0.846996  1.744497   

                        
F3         1         2  
F1         2         2  
F2                      
0  -2.751583       NaN  
1   0.923548  1.856369

更进一步的，stack的功能是和unstack对应，把column上的多级索引换到index上去

In [48]:
hierindexDF_gbsum.unstack([2, 0]).stack([1, 2])

F4
F2 F3 F1          
0  0  0   2.074013
      1   0.049578
      2  -0.952434
   1  0   1.849194
      1  -1.997105
      2  -2.751583
   2  0   1.998952
      1  -0.818087
1  0  0   0.159661
      1   0.768738
      2   1.744497
   1  0  -1.349574
      1   2.950726
      2   0.923548
   2  0  -0.687127
      1  -0.846996
      2   1.856369

本节完。